In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="darkgrid")

In [2]:
df = pd.read_csv('input/20190215LPS_CFD.csv', sep=';', usecols=[9,11,18], parse_dates=[1,2], encoding='iso8859_15')

In [3]:
df.head()

,Linux Plattform weiterentwickeln,2018-08-08,2018-08-15
0,Linux Plattform weiterentwickeln,2018-08-10,2018-08-10
1,Security,2018-07-20,2018-08-07
2,Berechtigung verwalten,2018-08-13,2018-08-13
3,Linux patchen updaten,2018-08-06,2018-08-07
4,Linux patchen updaten,2018-08-06,2018-08-09


In [4]:
df.dtypes

Linux Plattform weiterentwickeln            object
2018-08-08                          datetime64[ns]
2018-08-15                          datetime64[ns]
dtype: object

In [5]:
df.columns = ['art', 'input', 'out']
df.art.drop_duplicates()

0        Linux Plattform weiterentwickeln
1                                Security
2                  Berechtigung verwalten
3                   Linux patchen updaten
6                   Übergabe alter Themen
9      Linux System zur Verfügung stellen
22                         VMWare SAN LUN
23                                    ITR
44                       VMWare verwalten
139                             Migration
237                              Incident
Name: art, dtype: object

In [6]:
df.out - df.input 

0       0 days
1      18 days
2       0 days
3       1 days
4       3 days
5      17 days
6      11 days
7       5 days
8       0 days
9       2 days
10      1 days
11      3 days
12     18 days
13      2 days
14      1 days
15      7 days
16      1 days
17      2 days
18      1 days
19      1 days
20      4 days
21      5 days
22      1 days
23      0 days
24     17 days
25      8 days
26     17 days
27      3 days
28      7 days
29     12 days
        ...   
273    35 days
274     1 days
275     7 days
276     4 days
277     3 days
278    18 days
279     1 days
280    12 days
281     3 days
282    28 days
283    14 days
284     0 days
285     2 days
286     7 days
287     2 days
288     1 days
289     1 days
290   192 days
291     6 days
292   167 days
293   192 days
294     9 days
295    13 days
296    13 days
297    79 days
298    92 days
299    33 days
300     4 days
301    23 days
302     7 days
Length: 303, dtype: timedelta64[ns]

In [7]:
feiertage = ['2018-10-03', # Tag der Einheit
             '2018-11-01', # Allerheiligen
             '2018-12-24', # Heiligabend
             '2018-12-25', # Weihnachten
             '2018-12-26', # Weihnachten
            ]

# astype('datetime64[D]') notwendig
dauer=np.busday_count(df.input.values.astype('datetime64[D]'), 
                  df.out.values.astype('datetime64[D]'), 
                    holidays=feiertage)+1
dauer

array([  1,  13,   1,   2,   4,  13,   8,   4,   1,   3,   2,   4,  13,
         3,   2,   6,   2,   3,   2,   2,   5,   4,   2,   1,  14,   7,
        14,   4,   6,   9,   6,   2,   7,   6,   9,  10,   7,   3,   3,
         7,   1,  14,  22,  12,  35,  12,   3,   7,   5,   2,   5,   1,
        26,  12,  22,   3,   2,   2,   1,   1,   6,   1,   1,   1,   3,
         2,   3,   3,   2,  34,   1,  35,  11,   1,   5,   6,   2,   3,
         4,   3,   2,   1,   9,   2,   4,   8,   4,   2,   2,   2,   2,
         8,  44,   4,  29,   8,   5,   3,  10,   2,   4,  11,  10,   2,
         2,   9,  21,  32,   3,   3,   8,   1,   1,   7,   1,   3,   3,
         1,   2,   6,  13,   2,  28,   2,   2,  11,  11,   1,  90,   3,
        13,   7,  14,   2,   2,  13,  11,  35,   6,  22,   2,  16,   3,
         6,   3,   6,   4,   6,   7,   3,   1,   2,   9,   5,   2,   4,
         3,   9,  45,   4,   3,   3,   3,   4,   7,   3,  15,   3,   6,
         7,  18,   3,  17,   3,  14,  12,   8,  27,   2,   3,   

In [8]:
df['dauer'] = dauer

In [9]:
von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

def leadtime(ldf, art, von, bis):
    #interpolation noch überprüfen
    perc80 = np.percentile(ldf.dauer,80,interpolation='higher')
    perc90 = np.percentile(ldf.dauer,90,interpolation='higher')
    
    # gca stands for 'get current axis'
    ax = plt.gca()
    #hist zählt direkt die Elemente
    ldf.plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, label='')
    #df[gewaehlt].plot(kind='hist', y='dauer',bins=max(dauer),rwidth=1, color='black', ax=ax, title='selected work item type')

    text90 = '90% fertig in ' + str(perc90) + ' Tagen'
    
    # vertical dotted line originating at mean value
    plt.axvline(perc90+.1, linestyle='--', linewidth=1, color='red', label=text90)

    text80 = '80% fertig in ' + str(perc80) + ' Tagen'
    
    # vertical dotted line originating at mean value
    plt.axvline(perc80+.1, linestyle='-.', linewidth=1, color='blue', label=text80)

    # Beschriftung
    plt.suptitle(art)
    #raus = str(von) +' bis ' + str(bis) + ': ' + str(len(ldf) + ' Zettel'
    title = von +' bis ' + bis + ': ' + str(len(ldf)) + ' Zettel'
    plt.title(title)
    plt.xlabel('Dauer in Tagen')
    plt.ylabel('Anzahl')
    plt.legend(loc=0)
    
    #plt.show()
    filename='out/'+ art +'.png'
    plt.savefig(filename, dpi=300)
    plt.close()
#enddef plotte()

von=(min(df.input))
von=von.strftime('%d.%m.%Y')

bis=(max(df.out))
bis=bis.strftime('%d.%m.%Y')

for gew_art in df.art.drop_duplicates():
    gewaehlt = df.art == gew_art
#   gewaehlt = df.input > '2018-12-31'
    leadtime(df[gewaehlt], gew_art, von, bis)
#endfor gew_art in ....

leadtime(df, 'Alle', von, bis)

In [10]:
gewaehlt = ~(df.art == gew_art)
gewaehlt

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
273    True
274    True
275    True
276    True
277    True
278    True
279    True
280    True
281    True
282    True
283    True
284    True
285    True
286    True
287    True
288    True
289    True
290    True
291    True
292    True
293    True
294    True
295    True
296    True
297    True
298    True
299    True
300    True
301    True
302    True
Name: art, Length: 303, dtype: bool

In [11]:
df['art'].drop_duplicates()

0        Linux Plattform weiterentwickeln
1                                Security
2                  Berechtigung verwalten
3                   Linux patchen updaten
6                   Übergabe alter Themen
9      Linux System zur Verfügung stellen
22                         VMWare SAN LUN
23                                    ITR
44                       VMWare verwalten
139                             Migration
237                              Incident
Name: art, dtype: object